In [267]:
import emcee
import corner
import numpy as np
import matplotlib.pyplot as plt
import os.path
import pandas as pd
hvlist = np.loadtxt('hvlist.txt')
f = open(f'namelist.txt', "r", encoding="utf-8")
str1 = f.read()
namelist = str1.split()
mcmcdir = 'mcmc6'

建立一个 带空格的列表

In [268]:
name_space = []
for i in namelist:
    name_space.append(i[:4]+' '+i[4:])

读取wise的数据

In [269]:
df0 = pd.read_excel('mcwise.xlsx')
df0[:5]

,name,Dwise,Derr,albedo,Hv,P,other
0,1994 CB,193.0,10.0,0.2,NaN,8.676,NaN
1,1996 GQ,208.0,3.0,0.2,NaN,NaN,NaN
2,2002 LT24,143.0,24.0,0.2,NaN,NaN,NaN
3,2003 MA3,86.0,2.0,0.2,NaN,NaN,NaN
4,2008 NP3,193.0,3.0,0.2,23.3,NaN,NaN


生成当前目录计算结果的 列表

In [270]:
anslist = []
noans = []
print('No ans :')
for i in range(len(namelist)):
    name = namelist[i]
    Hv = hvlist[i]
    if not os.path.exists(f'./{mcmcdir}/ansfit/fit.txt.{name}'):
        print(name_space[i])
        noans.append(name)
        continue
    ans = np.loadtxt(f'./{mcmcdir}/ansfit/fit.txt.{name}')
    eta,eta1,eta2 = ans[0]
    D,D1,D2 = ans[1]
    wf,wf1,wf2 = ans[2]
    pv = ans[3][1]
    anslist.append([name_space[i],eta,wf,pv,D,D+D1,D-D2])
noans = np.array(noans)
np.savetxt(f'{mcmcdir}_nan.txt',noans,fmt='%s')

No ans :
2003 MA3
2008 NP3
2010 EX11


转换当前list为dataframe

In [271]:
df = pd.DataFrame(anslist,columns=['name','eta','wf','pv','D','Dup','Ddown'])
df = df.round({'eta':2,'D':1,'wf':2,'pv':2,'Dup':1,'Ddown':1})
#df.to_excel(f'{mcmcdir}.xlsx',index=False)
df[:5]

,name,eta,wf,pv,D,Dup,Ddown
0,1994 CB,5.00,0.0,192.63,192.6,192.9,192.4
1,1996 GQ,2.60,0.0,139.61,139.6,140.4,138.9
2,2002 LT24,5.00,0.0,190.35,190.3,190.5,190.2
3,2009 UX17,5.00,0.0,309.33,309.3,309.5,309.2
4,2010 AJ30,4.97,0.0,150.56,150.6,151.2,150.1


合并wise的结果和计算结果

In [272]:
df.insert(loc=len(df.columns),column='D_wise',value=0)
df.insert(loc=len(df.columns),column='D_wiseErr',value=0)
df.insert(loc=4,column='pv_wise',value=0)
df[:5]

,name,eta,wf,pv,pv_wise,D,Dup,Ddown,D_wise,D_wiseErr
0,1994 CB,5.00,0.0,192.63,0,192.6,192.9,192.4,0,0
1,1996 GQ,2.60,0.0,139.61,0,139.6,140.4,138.9,0,0
2,2002 LT24,5.00,0.0,190.35,0,190.3,190.5,190.2,0,0
3,2009 UX17,5.00,0.0,309.33,0,309.3,309.5,309.2,0,0
4,2010 AJ30,4.97,0.0,150.56,0,150.6,151.2,150.1,0,0


In [273]:
for i in range(len(df)):
    D  =df0[df0['name'] == df.loc[i,'name']]['Dwise']
    df.loc[i,'D_wise'] = float(D)
    Derr  =df0[df0['name'] == df.loc[i,'name']]['Derr']
    df.loc[i,'D_wiseErr'] = float(Derr)
    pv  =df0[df0['name'] == df.loc[i,'name']]['albedo']
    df.loc[i,'pv_wise'] = float(pv)

In [274]:
D_df = df[['name','D','D_wise','D_wiseErr']]
D_df.to_excel(f'{mcmcdir}_D.xlsx')

In [275]:
df.to_excel(f'{mcmcdir}.xlsx')